In [1]:
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
from datasets import load_dataset
from transformers import set_seed
from tqdm import tqdm

In [2]:

seed = 42
set_seed(seed)






# Corrección: cargar JSON local correctamente
dataset = load_dataset('json', data_files='./data/discourse_qa.json')
#remove columns that are not needed "question" and "answer"
dataset = dataset.remove_columns(["question", "answer"])
#split train and test
dataset = dataset["train"].train_test_split(0.1)

In [3]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# ejemplos planos
# predictions = ["respuesta generada por el modelo 1", "respuesta generada 2"]
# reales = ["respuesta real 1", "respuesta real 2"]

reales = [i['text'] for i in dataset['test']]

---

In [4]:
model_name = "PabloCano1/llama1b-entreno"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             attn_implementation='eager',
                                             device_map="cuda",dtype=torch.bfloat16)
# model.to("cuda")

In [6]:
predictions = []
reales_only_response = []
for real in tqdm(reales):
    real_no_response = real[:real.find('### Expected Response:') + len("### Expected Response:")].strip()
    inputs = tokenizer(real_no_response, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=None,
            do_sample=True,
            temperature=0.5,
            top_p=0.90,
            pad_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(f"{output=}")
    # print(f"{real=}")
    reales_only_response.append(real[real.find('### Expected Response:') + len('### Expected Response:'):].strip())
    predictions.append(output[output.find('### Expected Response:')+len('### Expected Response:'):].strip())
    # print(f"{reales_only_response[-1]=}")
    # print(f"{predictions[-1]=}")


100%|██████████| 884/884 [10:22<00:00,  1.42it/s]


In [7]:

# Para BLEU la librería espera referencias como lista de listas (cada pred puede tener varias refs)
references_for_bleu = [[r] for r in reales_only_response]   # [["respuesta real 1"], ["respuesta real 2"]]

# Opción 1: pasar cadenas y dejar que evaluate haga el tokenizado internamente
bleu_result = bleu.compute(predictions=predictions, references=references_for_bleu)
print("BLEU:", bleu_result["bleu"])

# ROUGE normalmente recibe listas de strings
rouge_result = rouge.compute(predictions=predictions, references=reales_only_response)
print("ROUGE:", rouge_result)

BLEU: 0.03018014538913631
ROUGE: {'rouge1': np.float64(0.3161894608405227), 'rouge2': np.float64(0.11906154346984041), 'rougeL': np.float64(0.30484769362609815), 'rougeLsum': np.float64(0.31318760950302515)}


In [8]:
prompt =reales[0][:reales[0].find('### Expected Response:') + len("### Expected Response:")]
continuacion_real = reales_only_response[0]
continuacion_pred = predictions[0]

In [15]:
# Tokeniza prompt y continuación
means = []
for ind in tqdm(range(len(reales))):
    prompt =reales[ind][:reales[ind].find('### Expected Response:') + len("### Expected Response:")]
    continuacion_real = reales_only_response[ind]
    continuacion_pred = predictions[ind]
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    continuation_ids = tokenizer(continuacion_pred, add_special_tokens=False, return_tensors="pt").input_ids.squeeze(0).to(model.device)

    log_prob_sum = 0.0
    with torch.no_grad():
        for i in range(continuation_ids.size(0)):
            context_ids = torch.cat([input_ids, continuation_ids[:i].unsqueeze(0)], dim=1).to(model.device)
            outputs = model(input_ids=context_ids)
            logits = outputs.logits[:, -1, :]          # Predicción del siguiente token
            log_probs = torch.log_softmax(logits, dim=-1)
            idx = continuation_ids[i].item()
            log_prob_sum += log_probs[0, idx].item()
    if continuation_ids.size(0) == 0:
        print(prompt)
        prob_media = 0.0
    else:
        prob_media = torch.exp(torch.tensor(log_prob_sum / continuation_ids.size(0))).item()
    means.append(prob_media)
    # print(f"Probabilidad media por token: {prob_media:.2f}")


  4%|▍         | 36/884 [01:52<42:31,  3.01s/it]  

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
okay .
 like West Mount area .

### Expected Response:


100%|██████████| 884/884 [51:01<00:00,  3.46s/it]  


In [17]:
np.mean(means)

np.float64(0.17826805992380662)

---

In [4]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             attn_implementation='eager',
                                             device_map="cuda",dtype=torch.bfloat16)

In [5]:
predictions = []
reales_only_response = []
for real in tqdm(reales):
    real_no_response = real[:real.find('### Expected Response:') + len("### Expected Response:")].strip()
    inputs = tokenizer(real_no_response, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=None,
            do_sample=True,
            # temperature=0.2,
            # top_p=0.90,
            pad_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(f"{output=}")
    # print(f"{real=}")
    reales_only_response.append(real[real.find('### Expected Response:') + len('### Expected Response:'):].strip())
    predictions.append(output[output.find('### Expected Response:')+len('### Expected Response:'):].strip())
    # print(f"{reales_only_response[-1]=}")

100%|██████████| 884/884 [10:22<00:00,  1.42it/s]


In [6]:

# Para BLEU la librería espera referencias como lista de listas (cada pred puede tener varias refs)
references_for_bleu = [[r] for r in reales_only_response]   # [["respuesta real 1"], ["respuesta real 2"]]

# Opción 1: pasar cadenas y dejar que evaluate haga el tokenizado internamente
bleu_result = bleu.compute(predictions=predictions, references=references_for_bleu)
print("BLEU:", bleu_result["bleu"])

# ROUGE normalmente recibe listas de strings
rouge_result = rouge.compute(predictions=predictions, references=reales_only_response)
print("ROUGE:", rouge_result)

BLEU: 0.0019859048546859376
ROUGE: {'rouge1': np.float64(0.055644729371710985), 'rouge2': np.float64(0.009380320904948525), 'rougeL': np.float64(0.04835310189850546), 'rougeLsum': np.float64(0.052667108418732254)}


---

In [13]:
prompt =reales[0][:reales[0].find('### Expected Response:') + len("### Expected Response:")]
continuacion_real = reales_only_response[0]
continuacion_pred = predictions[0]

In [7]:
# Tokeniza prompt y continuación
means = []
for ind in tqdm(range(len(reales))):
    prompt =reales[ind][:reales[ind].find('### Expected Response:') + len("### Expected Response:")]
    continuacion_real = reales_only_response[ind]
    continuacion_pred = predictions[ind]
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    continuation_ids = tokenizer(continuacion_pred, add_special_tokens=False, return_tensors="pt").input_ids.squeeze(0).to(model.device)

    log_prob_sum = 0.0
    with torch.no_grad():
        for i in range(continuation_ids.size(0)):
            context_ids = torch.cat([input_ids, continuation_ids[:i].unsqueeze(0)], dim=1).to(model.device)
            outputs = model(input_ids=context_ids)
            logits = outputs.logits[:, -1, :]          # Predicción del siguiente token
            log_probs = torch.log_softmax(logits, dim=-1)
            idx = continuation_ids[i].item()
            log_prob_sum += log_probs[0, idx].item()
    if continuation_ids.size(0) == 0:
        print(prompt)
        prob_media = 0.0
    else:
        prob_media = torch.exp(torch.tensor(log_prob_sum / continuation_ids.size(0))).item()
    means.append(prob_media)

  1%|          | 8/884 [00:33<57:23,  3.93s/it]  

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
interesting .
 funny how that works sometimes .

### Expected Response:


  1%|▏         | 13/884 [00:47<45:59,  3.17s/it]

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
mhm .

### Expected Response:


  2%|▏         | 20/884 [01:11<51:28,  3.58s/it]

...

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
there you go .

### Expected Response:
You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
wow .
 and it's always kind of Peter Pan .

### Expected Response:


 98%|█████████▊| 864/884 [45:31<01:02,  3.11s/it]

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
&-um and I know that you like to travel .

### Expected Response:


100%|██████████| 884/884 [46:36<00:00,  3.16s/it]

You are a patient that has gone to do an interview with a psychologist. The psychologist will ask you a series of questions and you will answer them in a natural way:

### Input:
total classic .
 yeah .
 one that I've not actually seen but I know that a lot of people have .

### Expected Response:


In [8]:
np.mean(means)

np.float64(0.1432432063359544)